# Multi-Agent RAG System Using Crew AI for PDF Question Answering

In [ ]:
from crewai import Agent, Crew, Process, Task, LLM
from crewai_tools import JSONSearchTool, PDFSearchTool
import os


In [2]:
llm=LLM(model="ollama/llama3.1", base_url="http://localhost:11434")

In [3]:
pdf_search_tool = PDFSearchTool(
    pdf = "AIAct.pdf",
    config = dict(
        llm = dict(
            provider = "ollama",
            config = dict(
                model = "llama3.1"
            )
        ),
        embedder = dict(
            provider = "ollama",
            config = dict(
                model = "llama3.1"
            )
        )
    )
)

Inserting batches in chromadb: 100%|██████████| 7/7 [05:26<00:00, 46.61s/it]


In [4]:
research_agent = Agent(
    role="Research Agent",
    goal="Search through the PDF file to find relevant answers",
    allow_delegation=False,
    verbose=True,
    llm=llm,
    backstory=(
        """
        The research agent is adept at searching and extracting data from documents,
        ensuring accurate and prompt responses.
        """
    ),
    tools=[pdf_search_tool],
)

In [5]:
response_writer_agent = Agent(
    role="Response Writer",
    goal="Write clear and concise responses based on the research agent's findings",
    allow_delegation=False,
    verbose=True,
    llm=llm,
    backstory=(
        """
        The response writer agent has excellent writing skills and is able to craft
        clear and concise responses based on the provided information.
        """
    ),
    tools=[],
)

In [6]:
analyse_data_task = Task(
    description=(
        """
        Use the PDF search tool to find relevant information about AI Act.
        Based on this information, write a response to the following question:
        {user_question}
        """
    ),
    expected_output="""
        A clear and concise response to the customer's question, based on the content
        of the AI ACT PDF file.
        """,
    tools=[pdf_search_tool],
    agent=research_agent,
)

In [7]:
write_response_task = Task(
    description=(
        """
        Write a clear and concise response to the following question, based on the
        information provided by the research agent:
        {user_question}
        """
    ),
    expected_output="""
        A clear and concise response to the customer's question, based on the
        information provided by the research agent.
        """,
    tools=[],
    agent=response_writer_agent,
)

In [8]:
crew = Crew(
    agents=[research_agent, response_writer_agent],
    tasks=[analyse_data_task, write_response_task],
    process=Process.sequential,
)

In [9]:
user_question = "What are the guidelines for the use of AI in biometric identification?"

In [10]:
result = crew.kickoff(
    inputs={"user_question": user_question}
)



> Entering new CrewAgentExecutor chain...
Thought: I need to search the PDF file for information about AI Act.

Action: Search a PDF's content
Action Input: {'query': 'AI Act 

Relevant Content:
information is valuable for the AI Office to anticipate the placing on the market of general-pur pose AI models with syste mic risks and the provider s can start to engag e with the AI Office early on. That information is especially OJ L, 12.7.2024 EN ELI: http:/data.europa.eu/eli/reg/2024/1689/oj 29/144

Regulation should comply with the relevant procedural requirements set by the Regulation (EC) No 810/2009 of the European Parliament and OJ L, 12.7.2024 EN ELI: http:/data.europa.eu/eli/reg/2024/1689/oj 17/144

influenced by conditions of misuse, model reliability , model fairness and model secur ity, the level of autonom y of EN OJ L, 12.7.2024 28/144 ELI: http:/data.europa.eu/eli/reg/2024/1689/oj

Thought:
Thought: I need to search the PDF file for information about guidelines for AI in bi

In [11]:
print(result)

In accordance with Regulation (EU) 2024/1689, the guidelines for the use of AI in biometric identification are as follows:

1. The AI system's technical documentation must include information about its operating logic and any relevant data or inputs used by the system.
2. The status of the AI system must be clearly indicated, including whether it is currently on the market, in service, or no longer available.
3. A certificate issued by a notified body, if applicable, must be provided, along with a scanned copy of the certificate.
4. Information about any Member States where the AI system has been placed on the market, put into service, or made available within the Union must be supplied.
5. A copy of the EU declaration of conformity is required, as per Article 47.
6. Electronic instructions for use are mandatory, except in high-risk areas such as law enforcement, migration, asylum, and border control management (as specified in Annex III).
7. An optional URL for additional information 